# సెమాంటిక్ కర్నెల్ OpenBnB MCP సర్వర్ ఇంటిగ్రేషన్‌తో

ఈ నోట్‌బుక్ MCPStdioPlugin ఉపయోగించి నిజమైన OpenBnB MCP సర్వర్‌తో సెమాంటిక్ కర్నెల్‌ను ఉపయోగించి నిజమైన Airbnb వసతులను శోధించడం ఎలా చేయాలో చూపిస్తుంది. LLM యాక్సెస్ కోసం, ఇది Azure AI Foundryని ఉపయోగిస్తుంది. మీ ఎన్విరాన్‌మెంట్ వేరియబుల్స్ సెట్ చేయడానికి, మీరు [సెట్టప్ పాఠం](/00-course-setup/README.md)ను అనుసరించవచ్చు.


## అవసరమైన ప్యాకేజీలను దిగుమతి చేసుకోండి


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP ప్లగిన్ కనెక్షన్ సృష్టించడం

మేము [OpenBnB MCP సర్వర్](https://github.com/openbnb-org/mcp-server-airbnb) కు MCPStdioPlugin ఉపయోగించి కనెక్ట్ అవుతాము. ఈ సర్వర్ @openbnb/mcp-server-airbnb ప్యాకేజీ ద్వారా Airbnb శోధన ఫంక్షనాలిటీని అందిస్తుంది.


## క్లయింట్ సృష్టించడం

ఈ ఉదాహరణలో, మనం మన LLM యాక్సెస్ కోసం Azure AI Foundry ను ఉపయోగిస్తాము. మీ ఎన్విరాన్‌మెంట్ వేరియబుల్స్ సరిగా సెట్ చేయబడ్డాయో లేదో నిర్ధారించుకోండి.


## పరిసరాల కాన్ఫిగరేషన్

Azure OpenAI సెట్టింగులను కాన్ఫిగర్ చేయండి. మీరు క్రింది పరిసర వేరియబుల్స్ సెట్ చేసి ఉండాలి:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP ఇంటిగ్రేషన్‌ను అర్థం చేసుకోవడం

ఈ నోట్‌బుక్ **నిజమైన OpenBnB MCP సర్వర్** కు కనెక్ట్ అవుతుంది, ఇది అసలు Airbnb శోధన ఫంక్షనాలిటీని అందిస్తుంది.

### ఇది ఎలా పనిచేస్తుంది:

1. **MCPStdioPlugin**: MCP సర్వర్‌తో స్టాండర్డ్ ఇన్‌పుట్/ఔట్‌పుట్ కమ్యూనికేషన్‌ను ఉపయోగిస్తుంది
2. **నిజమైన NPM ప్యాకేజీ**: `@openbnb/mcp-server-airbnb` ను npx ద్వారా డౌన్‌లోడ్ చేసి నడుపుతుంది
3. **లైవ్ డేటా**: వారి APIs నుండి అసలు Airbnb ప్రాపర్టీ డేటాను అందిస్తుంది
4. **ఫంక్షన్ డిస్కవరీ**: MCP సర్వర్ నుండి అందుబాటులో ఉన్న ఫంక్షన్లను ఏజెంట్ ఆటోమేటిక్‌గా కనుగొంటుంది

### అందుబాటులో ఉన్న ఫంక్షన్లు:

OpenBnB MCP సర్వర్ సాధారణంగా అందిస్తుంది:
- **search_listings** - ప్రదేశం మరియు ప్రమాణాల ఆధారంగా Airbnb ప్రాపర్టీలను శోధించండి
- **get_listing_details** - నిర్దిష్ట ప్రాపర్టీల గురించి వివరమైన సమాచారం పొందండి
- **check_availability** - నిర్దిష్ట తేదీలకు అందుబాటును తనిఖీ చేయండి
- **get_reviews** - ప్రాపర్టీలకు సమీక్షలను పొందండి
- **get_host_info** - ప్రాపర్టీ హోస్టుల గురించి సమాచారం పొందండి

### అవసరమైనవి:

- మీ సిస్టమ్‌లో **Node.js** ఇన్‌స్టాల్ చేయబడాలి
- **ఇంటర్నెట్ కనెక్షన్** MCP సర్వర్ ప్యాకేజీని డౌన్‌లోడ్ చేయడానికి
- **NPX** అందుబాటులో ఉండాలి (ఇది Node.js తో వస్తుంది)

### కనెక్షన్‌ను పరీక్షించడం:

మీరు MCP సర్వర్‌ను మాన్యువల్‌గా పరీక్షించవచ్చు, ఇలా నడిపి:
```bash
npx -y @openbnb/mcp-server-airbnb
```

ఇది OpenBnB MCP సర్వర్‌ను డౌన్‌లోడ్ చేసి ప్రారంభిస్తుంది, దీని తర్వాత Semantic Kernel నిజమైన Airbnb డేటాకు కనెక్ట్ అవుతుంది.


## OpenBnB MCP సర్వర్‌తో ఏజెంట్‌ను నడపడం

ఇప్పుడు మనం OpenBnB MCP సర్వర్‌కు కనెక్ట్ అయ్యే AI ఏజెంట్‌ను నడుపుతాము, ఇది 2 మంది పెద్దలు మరియు 1 పిల్లవాడి కోసం స్టాక్‌హోమ్‌లో నిజమైన Airbnb వసతులను వెతుకుతుంది. శోధన ప్రమాణాలను మార్చడానికి `user_inputs` జాబితాను స్వేచ్ఛగా మార్చండి.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# సారాంశం  
అభినందనలు! మీరు Model Context Protocol (MCP) ఉపయోగించి నిజజీవిత వసతి శోధనతో అనుసంధానమయ్యే AI ఏజెంట్‌ను విజయవంతంగా నిర్మించారు:

## ఉపయోగించిన సాంకేతికతలు:  
- సెమాంటిక్ కర్నెల్ - Azure OpenAIతో తెలివైన ఏజెంట్లను నిర్మించడానికి  
- Azure AI Foundry - LLM సామర్థ్యాలు మరియు చాట్ పూర్తి చేయడానికి  
- MCP (మోడల్ కాంటెక్స్ట్ ప్రోటోకాల్) - ప్రామాణిక టూల్ ఇంటిగ్రేషన్ కోసం  
- OpenBnB MCP సర్వర్ - నిజమైన Airbnb శోధన ఫంక్షనాలిటీ కోసం  
- Node.js/NPX - బాహ్య MCP సర్వర్‌ను నడపడానికి  

## మీరు నేర్చుకున్నవి:  
- MCP ఇంటిగ్రేషన్: సెమాంటిక్ కర్నెల్ ఏజెంట్లను బాహ్య MCP సర్వర్లకు అనుసంధానించడం  
- రియల్-టైమ్ డేటా యాక్సెస్: ప్రత్యక్ష APIల ద్వారా నిజమైన Airbnb ప్రాపర్టీలను శోధించడం  
- ప్రోటోకాల్ కమ్యూనికేషన్: ఏజెంట్ మరియు MCP సర్వర్ మధ్య stdio కమ్యూనికేషన్ ఉపయోగించడం  
- ఫంక్షన్ డిస్కవరీ: MCP సర్వర్ల నుండి అందుబాటులో ఉన్న ఫంక్షన్లను ఆటోమేటిక్‌గా కనుగొనడం  
- స్ట్రీమింగ్ రెస్పాన్స్‌లు: ఫంక్షన్ కాల్స్‌ను రియల్-టైమ్‌లో క్యాప్చర్ చేసి లాగ్ చేయడం  
- HTML రెండరింగ్: ఏజెంట్ రెస్పాన్స్‌లను స్టైల్డ్ టేబుల్స్ మరియు ఇంటరాక్టివ్ డిస్ప్లేలతో ఫార్మాట్ చేయడం  

## తదుపరి దశలు:  
- అదనపు MCP సర్వర్లను (వాతావరణం, విమానాలు, రెస్టారెంట్లు) అనుసంధానించండి  
- MCP మరియు A2A ప్రోటోకాల్‌లను కలిపి మల్టీ-ఏజెంట్ సిస్టమ్‌ను నిర్మించండి  
- మీ స్వంత డేటా సోర్స్‌ల కోసం కస్టమ్ MCP సర్వర్లను సృష్టించండి  
- సెషన్‌ల మధ్య స్థిరమైన సంభాషణ జ్ఞాపకశక్తిని అమలు చేయండి  
- MCP సర్వర్ ఆర్కెస్ట్రేషన్‌తో ఏజెంట్‌ను Azure ఫంక్షన్లకు డిప్లాయ్ చేయండి  
- యూజర్ ఆథెంటికేషన్ మరియు బుకింగ్ సామర్థ్యాలను జోడించండి  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**విమర్శ**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నిస్తున్నప్పటికీ, ఆటోమేటెడ్ అనువాదాలలో తప్పులు లేదా అసమగ్రతలు ఉండవచ్చు. దయచేసి, మూల భాషలోని అసలు పత్రాన్ని అధికారం కలిగిన మూలంగా పరిగణించండి. ముఖ్యమైన సమాచారం కోసం, ప్రొఫెషనల్ మానవ అనువాదాన్ని సిఫారసు చేస్తాము. ఈ అనువాదాన్ని ఉపయోగించడం వల్ల కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారులు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
